In [12]:
#r "nuget: Microsoft.ML"
#r "nuget: Microsoft.SemanticKernel"

Installed Packages Microsoft.ML, 4.0.2 Microsoft.SemanticKernel, 1.66.0

## 1. Import thư viện cần thiết

Đầu tiên, chúng ta cần import các namespace từ ML.NET để sử dụng các chức năng machine learning.

In [13]:
// Import namespace chính của ML.NET để sử dụng MLContext và các chức năng cốt lõi
using Microsoft.ML;

// Import namespace cho các attribute để định nghĩa cấu trúc data
using Microsoft.ML.Data;

## 2. Định nghĩa class cho dữ liệu đầu vào

`SentimentData` là class đại diện cho dữ liệu training, bao gồm text cần phân tích và nhãn cảm xúc tương ứng.

In [14]:
public class SentimentData
{
    // LoadColumn(0): Chỉ định rằng property này sẽ được load từ cột 0 của file dữ liệu
    [LoadColumn(0)]
    public string Text { get; set; }

    // LoadColumn(1): Load từ cột 1 của file dữ liệu
    // ColumnName("Label"): Đặt tên cột trong ML pipeline là "Label"
    [LoadColumn(1), ColumnName("Label")]
    public bool Sentiment { get; set; } // true = positive, false = negative
}

## 3. Định nghĩa class cho kết quả dự đoán

`SentimentPrediction` chứa kết quả dự đoán từ mô hình, bao gồm nhãn dự đoán, xác suất và điểm số.

In [15]:
public class SentimentPrediction
{
    // ColumnName("PredictedLabel"): Map với cột "PredictedLabel" từ output của mô hình
    [ColumnName("PredictedLabel")]
    public bool Prediction { get; set; } // Kết quả dự đoán: true = positive, false = negative
    
    // Xác suất của prediction (từ 0.0 đến 1.0)
    public float Probability { get; set; }
    
    // Điểm số raw từ mô hình (có thể âm hoặc dương)
    public float Score { get; set; }
}

## 4. Khởi tạo ML Context

`MLContext` là entry point chính cho tất cả các hoạt động ML.NET. Nó cung cấp các API để load dữ liệu, tạo pipeline và train mô hình.

In [16]:
// Tạo một instance của MLContext - đây là điểm bắt đầu cho mọi thao tác ML.NET
var mlContext = new MLContext();

## 5. Load dữ liệu training

Chúng ta load dữ liệu từ file TSV (Tab-Separated Values) và chuyển đổi thành format mà ML.NET có thể hiểu được.

In [17]:
// LoadFromTextFile: Load dữ liệu từ file text
// <SentimentData>: Chỉ định kiểu dữ liệu để ML.NET biết cách parse
// "data.tsv": Đường dẫn đến file dữ liệu
// hasHeader: true: File có header ở dòng đầu tiên
var data = mlContext.Data.LoadFromTextFile<SentimentData>(
    "data.tsv", hasHeader: true);

## 6. Tạo Machine Learning Pipeline

Pipeline định nghĩa các bước xử lý dữ liệu và thuật toán machine learning sẽ được sử dụng.

In [18]:
// Tạo pipeline với 2 bước:
var pipeline = mlContext.Transforms.Text
    // Bước 1: FeaturizeText - Chuyển đổi text thành vector số
    // "Features": Tên cột output chứa features
    // nameof(SentimentData.Text): Sử dụng cột Text từ SentimentData làm input
    .FeaturizeText("Features", nameof(SentimentData.Text))
    
    // Bước 2: Append thuật toán SdcaLogisticRegression
    // Đây là thuật toán binary classification phù hợp cho sentiment analysis
    .Append(mlContext.BinaryClassification.Trainers.SdcaLogisticRegression());

## 7. Train mô hình

Sử dụng dữ liệu training để train mô hình theo pipeline đã định nghĩa.

In [19]:
// Fit: Train mô hình sử dụng dữ liệu training
// Kết quả là một mô hình đã được train và sẵn sàng để predict
var model = pipeline.Fit(data);

## 8. Tạo Prediction Engine

Prediction Engine là wrapper giúp thực hiện prediction dễ dàng hơn cho single prediction.

In [20]:
// CreatePredictionEngine: Tạo engine để thực hiện prediction
// <SentimentData, SentimentPrediction>: Chỉ định kiểu input và output
var engine = mlContext.Model.CreatePredictionEngine<SentimentData, SentimentPrediction>(model);

## 9. Test mô hình với dữ liệu mẫu

Thực hiện prediction trên một câu sample để kiểm tra mô hình hoạt động.

In [21]:
// Tạo dữ liệu sample để test
// Câu này có vẻ negative: "Total waste of time and money"
var sample = new SentimentData { Text = "Total waste of time and money" };

// Predict: Thực hiện dự đoán cảm xúc cho sample
var prediction = engine.Predict(sample);

// In kết quả:
// Prediction: true/false (positive/negative)
// Score: Điểm số raw từ mô hình
Console.WriteLine($"Positive: {prediction.Prediction}, Score: {prediction.Score}");

Positive: False, Score: -2.2028353
